In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.discriminant_analysis import StandardScaler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
%pip install lightgbm


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
random_state=42
best_models = {}

In [6]:
# Read in the data
df = pd.read_csv('../Data/hashed_combined.csv')
df.drop(['fitspatrick','region_hash_0','region_hash_1','region_hash_2','region_hash_3','region_hash_4','region_hash_5','region_hash_6','region_hash_7','region_hash_8','region_hash_9','changed','bleed','elevation','biopsed','diameter_2','diameter_1','itch','grew','hurt'], axis=1, inplace=True)
df

,smoke,drink,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,has_sewage_system,diagnostic,...,background_mother_hash_0,background_mother_hash_1,background_mother_hash_2,background_mother_hash_3,background_mother_hash_4,background_mother_hash_5,background_mother_hash_6,background_mother_hash_7,background_mother_hash_8,background_mother_hash_9
0,False,False,55,False,0,True,True,True,True,1,...,1.0,-1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,False,True,79,False,1,True,False,False,False,1,...,1.0,-1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,False,True,52,False,0,False,True,True,True,1,...,0.0,-2.0,1.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0
3,False,False,74,True,0,False,False,False,False,1,...,1.0,-1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,False,True,58,True,0,True,True,True,True,1,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1700,False,False,23,True,0,False,True,True,True,0,...,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-1.0,0.0
1701,False,False,27,False,0,False,False,True,True,0,...,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-1.0,0.0
1702,True,True,23,False,1,False,False,True,True,0,...,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-1.0,0.0
1703,False,False,23,True,0,False,False,True,False,0,...,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-1.0,0.0


# New Section

In [7]:
from imblearn.under_sampling import ClusterCentroids, TomekLinks
from imblearn.over_sampling import RandomOverSampler

def splitting_data(df, sampling):
    X = df.drop(['diagnostic'], axis=1)
    y = df['diagnostic']

    if sampling == 'none':
        return X, y
    elif sampling == 'SMOTEENN':
        sampler = SMOTEENN(random_state=random_state)
    elif sampling == 'SMOTE':
        sampler = SMOTE(random_state=random_state)
    elif sampling == 'under':
        sampler = RandomUnderSampler(random_state=random_state)
    elif sampling == 'over':
        sampler = RandomOverSampler(random_state=random_state)
    elif sampling == 'cluster_centroids':
        sampler = ClusterCentroids(random_state=random_state)
    elif sampling == 'tomek_links':
        sampler = TomekLinks()

    X_resampled, y_resampled = sampler.fit_resample(X, y)
    return X_resampled, y_resampled


In [8]:

def training(X_train, y_train):
    # Create a KNN classifier with 5 neighbors
    LGBM = ExtraTreesClassifier()
    # Fit the classifier to the data
    LGBM.fit(X_train, y_train)
    return LGBM

In [9]:
def best_model(modelName, accuracy, precision, recall, f1):
    best_models[modelName] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [10]:
def predict(modleName,LGBM, X_test ,y_test):
    # Predict the labels for the training data X
    y_pred = LGBM.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    cr=classification_report(y_test, y_pred, output_dict=True)
    precision = cr['weighted avg']['precision']
    recall = cr['weighted avg']['recall']
    f1 = cr['weighted avg']['f1-score']
    best_model(modleName,accuracy,precision,recall,f1)
    cr=classification_report(y_test, y_pred)
    print(cr)

In [11]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def optimize_with_grid(X_train, y_train):
    # Define a pipeline
    # Note: Scaling might not be necessary for tree-based models, but included for consistency
    pipe = Pipeline([
        ('scaler', StandardScaler()),  # Optional for ExtraTrees
        ('extra_trees', ExtraTreesClassifier(random_state=random_state))
    ])

    # Define the parameter grid to search
    param_grid = {
        'extra_trees__n_estimators': [100, 200, 300],  # Number of trees in the forest
        'extra_trees__max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
        'extra_trees__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
        'extra_trees__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
        'extra_trees__max_features': ['auto', 'sqrt', 'log2']  # Number of features to consider when looking for the best split
    }

    # Create the GridSearchCV object
    extra_trees_cv = GridSearchCV(pipe, param_grid, cv=5, verbose=1, n_jobs=-1)

    # Perform the grid search on the provided data
    extra_trees_cv.fit(X_train, y_train)

    # Best parameters and best score
    best_params = extra_trees_cv.best_params_
    best_score = extra_trees_cv.best_score_
    best_estimator = extra_trees_cv.best_estimator_
    print("Best Parameters:", best_params)
    print("Best Score:", best_score)

    return best_estimator

# Example usage
# Ensure you have defined X_train, y_train, and optionally random_state before calling this function
# best_extra_trees_model = optimize_with_grid_extra_trees(X_train, y_train, random_state=42)


<h1> LGBM on original data with optimization </h1>

In [12]:
# using function with no sampling
X, y= splitting_data(df, 'none')
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
1    1494
0     211
Name: diagnostic, dtype: int64


In [14]:
LGBM1 = training(X_train, y_train)
y_pred = predict('original',LGBM1, X_test, y_test)


              precision    recall  f1-score   support

           0       1.00      0.97      0.99        37
           1       1.00      1.00      1.00       304

    accuracy                           1.00       341
   macro avg       1.00      0.99      0.99       341
weighted avg       1.00      1.00      1.00       341



In [15]:
best_LGBM1 = optimize_with_grid(X_train, y_train)
prediction = predict('original_grid',best_LGBM1, X_test, y_test)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Parameters: {'extra_trees__max_depth': None, 'extra_trees__max_features': 'auto', 'extra_trees__min_samples_leaf': 1, 'extra_trees__min_samples_split': 2, 'extra_trees__n_estimators': 100}
Best Score: 0.9978021978021978
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        37
           1       1.00      1.00      1.00       304

    accuracy                           1.00       341
   macro avg       1.00      0.99      0.99       341
weighted avg       1.00      1.00      1.00       341



<h1> LGBM using SMOTE sampling </h1>

In [16]:
X,y = splitting_data(df, 'SMOTE')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
1    1494
0    1494
Name: diagnostic, dtype: int64


In [18]:
LGBM2 =training(X_train, y_train)
y_pred = predict('SMOTE',LGBM2, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       284
           1       1.00      1.00      1.00       314

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



In [19]:
best_LGBM2 = optimize_with_grid(X_train, y_train)
prediction = predict('SMOTE_grid',best_LGBM2, X_test, y_test)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Parameters: {'extra_trees__max_depth': None, 'extra_trees__max_features': 'auto', 'extra_trees__min_samples_leaf': 1, 'extra_trees__min_samples_split': 2, 'extra_trees__n_estimators': 100}
Best Score: 0.999581589958159
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       284
           1       1.00      1.00      1.00       314

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



<h1> LGBM using SMOTEENN sampling </h1>

In [20]:
X,y = splitting_data(df, 'SMOTEENN')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
0    1493
1    1481
Name: diagnostic, dtype: int64


In [22]:
LGBM3 =training(X_train, y_train)
y_pred = predict('SMOTEENN',LGBM3, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       304
           1       1.00      1.00      1.00       291

    accuracy                           1.00       595
   macro avg       1.00      1.00      1.00       595
weighted avg       1.00      1.00      1.00       595



In [23]:
# from joblib import dump
# dump(LGBM3,'/content/LGBM_SMOTEENN.joblib')

In [24]:
# from google.colab import drive
# drive.mount('/content/drive')

In [25]:
best_LGBM3 = optimize_with_grid(X_train, y_train)
prediction = predict('SMOTEENN_grid',best_LGBM3, X_test, y_test)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Parameters: {'extra_trees__max_depth': None, 'extra_trees__max_features': 'auto', 'extra_trees__min_samples_leaf': 1, 'extra_trees__min_samples_split': 2, 'extra_trees__n_estimators': 100}
Best Score: 0.9991587793011941
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       304
           1       1.00      1.00      1.00       291

    accuracy                           1.00       595
   macro avg       1.00      1.00      1.00       595
weighted avg       1.00      1.00      1.00       595



<h1> DT on Random undersampling </h1>

In [26]:
X,y = splitting_data(df, 'under')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
0    211
1    211
Name: diagnostic, dtype: int64


In [28]:
LGBM4 =training(X_train, y_train)
y_pred = predict('undersampling',LGBM4, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        50

    accuracy                           1.00        85
   macro avg       1.00      1.00      1.00        85
weighted avg       1.00      1.00      1.00        85



In [29]:
best_LGBM4 = optimize_with_grid(X_train, y_train)
prediction = predict('undersampling_grid',best_LGBM4, X_test, y_test)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Parameters: {'extra_trees__max_depth': None, 'extra_trees__max_features': 'log2', 'extra_trees__min_samples_leaf': 1, 'extra_trees__min_samples_split': 2, 'extra_trees__n_estimators': 100}
Best Score: 0.9911325724319578
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        50

    accuracy                           1.00        85
   macro avg       1.00      1.00      1.00        85
weighted avg       1.00      1.00      1.00        85



<h1> DT on Random Oversampling </h1>

In [30]:
X,y = splitting_data(df, 'over')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [31]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
1    1494
0    1494
Name: diagnostic, dtype: int64


In [32]:
LGBM5 =training(X_train, y_train)
y_pred = predict('oversampling',LGBM5, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       292
           1       1.00      1.00      1.00       306

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



In [33]:
best_LGBM5 = optimize_with_grid(X_train, y_train)
prediction = predict('oversampling_grid',best_LGBM5, X_test, y_test)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Parameters: {'extra_trees__max_depth': None, 'extra_trees__max_features': 'auto', 'extra_trees__min_samples_leaf': 1, 'extra_trees__min_samples_split': 2, 'extra_trees__n_estimators': 100}
Best Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       292
           1       1.00      1.00      1.00       306

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



<h1> DT on Cluster Centroids </h1>

In [34]:
X,y = splitting_data(df, 'cluster_centroids')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [35]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
0    211
1    211
Name: diagnostic, dtype: int64


In [36]:
LGBM6 =training(X_train, y_train)
y_pred = predict('cluster_centroids',LGBM6, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        50

    accuracy                           1.00        85
   macro avg       1.00      1.00      1.00        85
weighted avg       1.00      1.00      1.00        85



In [37]:
best_LGBM6 = optimize_with_grid(X_train, y_train)
prediction = predict('cluster_centroids_grid',best_LGBM6, X_test, y_test)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


KeyboardInterrupt: 

<h1> DT on Tomek Links </h1>

In [ ]:
X,y = splitting_data(df, 'tomek_links')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print("Number of observations in each class in the set:")
print(y.value_counts())

In [ ]:
LGBM7 =training(X_train, y_train)
y_pred = predict('tomek_links',LGBM7, X_test, y_test)

In [ ]:
best_LGBM7 = optimize_with_grid(X_train, y_train)
prediction = predict('tomek_links_grid',best_LGBM7, X_test, y_test)

In [ ]:
best_model_df = pd.DataFrame.from_dict(best_models, orient='index')
best_model_df.sort_values(by='accuracy', ascending=False, inplace=True)
best_model_df